## Homework
### 使用 OutputParser 优化 overall_chain 输出格式，区分 synopsis_chain 和 review_chain 的结果
- 作业内容的样例有变更：
- 课程内容是：
    - synopsis_chain：你是一位剧作家。根据戏剧的标题，你的任务是为该标题写一个简介。
    - review_chain：你是《纽约时报》的戏剧评论家。根据剧情简介，你的工作是为该剧撰写一篇评论。

- 作业内容是：
    - content_chain：你是一个游戏原画设计师，根据关键词生成一段内容描述信息。
    - keywords_list_chain：根据的文本内容，生成原画关键词列表。


In [1]:

import os
import openai
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.chains import SequentialChain
from langchain import OpenAI, PromptTemplate, LLMChain

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
output_parser = CommaSeparatedListOutputParser()

llm = OpenAI(temperature=0.9, max_tokens=500, openai_api_key=openai.api_key)

prompt_1 = PromptTemplate(template="你是一个游戏原画设计师，根据关键词生成一段内容描述信息，关键词：{keywords}",
                          input_variables=["keywords"])

prompt_2 = PromptTemplate(template="根据的文本内容，生成原画关键词列表。\n文本内容：{text_content} \n{output_prompt}",
                          input_variables=["text_content"],
                          partial_variables={"output_prompt": output_parser.get_format_instructions()})

content_chain = LLMChain(llm=llm, prompt=prompt_1, output_key="text_content", verbose=True)
keywords_list_chain = LLMChain(llm=llm, prompt=prompt_2, output_key="keywords_list", verbose=True)
se_chain = SequentialChain(chains=[content_chain, keywords_list_chain], input_variables=["keywords"],
                           output_variables=["text_content", "keywords_list"], verbose=True)

In [3]:
result_se_chain = se_chain({"keywords": "12岁的阳光少年"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一个游戏原画设计师，根据关键词生成一段内容描述信息，关键词：12岁的阳光少年

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
根据的文本内容，生成原画关键词列表。
文本内容：

作为一个游戏原画设计师，我设计出一个有趣可爱的12岁阳光少年角色。他充满活力，目光充满激情，拥有温暖的脸庞，宽容的模样，深情的眼神、英俊的身体曲线和健康的衣着，彰显了青春期少年的纯真和活力。他的角色设定反映出他充满了孩子般的天真、乐观的态度，乐于奉献、勇敢坚毅，注重友谊与义气。 
Your response should be a list of comma separated values, eg: `foo, bar, baz`

> Finished chain.

> Finished chain.


In [4]:
output_parser.parse(result_se_chain.get("text_content"))

['作为一个游戏原画设计师，我设计出一个有趣可爱的12岁阳光少年角色。他充满活力，目光充满激情，拥有温暖的脸庞，宽容的模样，深情的眼神、英俊的身体曲线和健康的衣着，彰显了青春期少年的纯真和活力。他的角色设定反映出他充满了孩子般的天真、乐观的态度，乐于奉献、勇敢坚毅，注重友谊与义气。']

In [5]:
output_parser.parse(result_se_chain.get("keywords_list"))

['阳光少年',
 '活力',
 '激情',
 '温暖脸庞',
 '宽容模样',
 '深情眼神',
 '英俊身体曲线',
 '健康衣着',
 '青春期少年',
 '纯真',
 '孩子般天真',
 '乐观态度',
 '奉献',
 '勇敢坚毅',
 '友谊',
 '义气']